## Project 4 Random Forest trial

- Opt for using last 5 years - start small and then branch out if time permitting
- Columns to drop:
    - TREND
    - Avg Age Years and Schooling
    - Trend Value
    - NMSCI
    - GAIN
    - GAIN CI
    - Non-Attempt
    - NonStated
    - Withdrawn
    - NOD NMS Base (last 3 columns are stupid!)
    - Band grading 1 - 6. Certain grades have different bands - based on the score.  Score by percentage.
        - can put in presentation the different band types and %'s
    - 5220 score for Percentils values are Spelling, Parental Education and Parental Occupation subgroups BLANK DATA. = WHY? National.... RACH to look further into this one.

In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline


## Loading and Preprocessing Loans Encoded Data

In [2]:
# Loading data
df_Naplan = pd.read_csv("Naplan_results.csv")
df_Naplan.head(5)

,CALENDAR_YEAR,YEAR_LEVEL,DOMAIN,STATE,SUBGROUP,ENROLLED_STUDENTS,MEAN,MEAN_CI,MEAN_SD,NMS,...,PCTL05,PCTL20,PCTL80,PCTL95,TREND,TREND_VALUE,NOD_MEAN_PREV,NOD_NMS_PREV,NOD_MEAN_BASE,NOD_NMS_BASE
0,2008,3,Grammar and Punctuation,ACT,All,4366.0,419.6,6.2,83.1,93.6,...,276.5,350.1,490.2,551.7,Flat,434.8,NaN,NaN,NaN,NaN
1,2008,3,Grammar and Punctuation,ACT,Parental Education: Bachelor,167.0,445.2,NaN,79.7,95.8,...,308.7,381.3,513.0,565.4,NaN,NaN,NaN,NaN,NaN,NaN
2,2008,3,Grammar and Punctuation,ACT,Parental Education: Certificate,72.0,396.1,NaN,91.0,90.0,...,240.9,319.4,476.6,546.1,NaN,NaN,NaN,NaN,NaN,NaN
3,2008,3,Grammar and Punctuation,ACT,Parental Education: Diploma,52.0,401.4,NaN,69.1,96.9,...,283.7,342.1,456.3,519.2,NaN,NaN,NaN,NaN,NaN,NaN
4,2008,3,Grammar and Punctuation,ACT,Parental Education: Not stated,4009.0,419.8,NaN,82.7,93.7,...,278.0,351.0,489.8,551.1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cols= df_Naplan.columns
cols

Index(['CALENDAR_YEAR', 'YEAR_LEVEL', 'DOMAIN', 'STATE', 'SUBGROUP',
       'ENROLLED_STUDENTS', 'MEAN', 'MEAN_CI', 'MEAN_SD', 'NMS', 'NMS_CI',
       'GAIN', 'GAIN_CI', 'EXEMPT', 'ABSENT', 'ASSESSED', 'WITHDRAWN',
       'NONATTEMPT', 'PARTICIPATION_RATE', 'PARTICIPATION_NUMBER',
       'NOT_STATED', 'AVERAGE_AGE', 'YEARS_OF_SCHOOLING', 'BANDCOL1',
       'BANDCOL2', 'BANDCOL3', 'BANDCOL4', 'BANDCOL5', 'BANDCOL6', 'PCTL05',
       'PCTL20', 'PCTL80', 'PCTL95', 'TREND', 'TREND_VALUE', 'NOD_MEAN_PREV',
       'NOD_NMS_PREV', 'NOD_MEAN_BASE', 'NOD_NMS_BASE'],
      dtype='object')

In [4]:
col_drop= ['NONATTEMPT','NMS_CI', 'NOT_STATED','AVERAGE_AGE', 'YEARS_OF_SCHOOLING','TREND', 'TREND_VALUE','WITHDRAWN','NOD_MEAN_PREV','NOD_NMS_PREV', 'NOD_MEAN_BASE', 'NOD_NMS_BASE','GAIN_CI', 'GAIN', 'MEAN_CI', 'MEAN', 'MEAN_CI', 'MEAN_SD', 'DOMAIN', 'PCTL05',
       'PCTL20', 'PCTL80', 'PCTL95']

In [5]:
df_Naplan.drop(columns=col_drop, axis=1, inplace=True)

In [6]:
df_Naplan = df_Naplan[df_Naplan['YEAR_LEVEL'] > 3]
df_Naplan = df_Naplan[df_Naplan['STATE'] == "AUS"]
df_Naplan

,CALENDAR_YEAR,YEAR_LEVEL,STATE,SUBGROUP,ENROLLED_STUDENTS,NMS,EXEMPT,ABSENT,ASSESSED,PARTICIPATION_RATE,PARTICIPATION_NUMBER,BANDCOL1,BANDCOL2,BANDCOL3,BANDCOL4,BANDCOL5,BANDCOL6
689,2008,5,AUS,All,271423.0,91.9,1.5,2.7,95.0,96.9,263126,6.5,11.0,21.3,26.4,19.9,13.4
690,2008,5,AUS,Parental Education: Bachelor,41041.0,96.9,1.2,1.7,96.8,98.0,40233,1.9,5.1,14.3,25.8,26.7,24.9
691,2008,5,AUS,Parental Education: Certificate,41794.0,92.0,1.4,1.9,96.4,97.8,40871,6.5,13.0,24.7,28.2,17.7,8.4
692,2008,5,AUS,Parental Education: Diploma,22002.0,94.5,1.4,1.9,96.5,97.9,21547,4.1,9.2,21.2,28.9,21.9,13.3
693,2008,5,AUS,Parental Education: Not stated,128634.0,91.8,1.2,3.5,94.9,96.2,123691,7.0,11.0,21.3,26.3,19.7,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57990,2022,9,AUS,Outer regional,25648.0,74.3,1.8,13.2,80.0,82.1,21067,23.9,22.5,24.0,17.9,7.4,2.5
57991,2022,9,AUS,Remote,3188.0,66.2,2.8,17.7,76.4,79.6,2537,31.0,21.2,21.4,15.1,6.7,1.7
57992,2022,9,AUS,Very remote,2016.0,34.4,1.0,47.6,47.7,49.4,996,64.6,15.4,10.7,5.8,1.9,0.6
57993,2022,9,AUS,Parental Education: Year 11,26319.0,62.1,4.2,19.4,71.3,76.1,20016,33.8,24.8,21.1,11.6,3.8,0.8


In [7]:
df_Naplan.isna().sum()

CALENDAR_YEAR           0
YEAR_LEVEL              0
STATE                   0
SUBGROUP                0
ENROLLED_STUDENTS       0
NMS                     0
EXEMPT                  0
ABSENT                  0
ASSESSED                0
PARTICIPATION_RATE      0
PARTICIPATION_NUMBER    0
BANDCOL1                0
BANDCOL2                0
BANDCOL3                0
BANDCOL4                0
BANDCOL5                0
BANDCOL6                0
dtype: int64

In [8]:
df_Naplan.dtypes

CALENDAR_YEAR             int64
YEAR_LEVEL                int64
STATE                    object
SUBGROUP                 object
ENROLLED_STUDENTS       float64
NMS                     float64
EXEMPT                  float64
ABSENT                  float64
ASSESSED                float64
PARTICIPATION_RATE      float64
PARTICIPATION_NUMBER      int64
BANDCOL1                float64
BANDCOL2                float64
BANDCOL3                float64
BANDCOL4                float64
BANDCOL5                float64
BANDCOL6                float64
dtype: object

In [9]:
df_Naplan_copy = df_Naplan.copy()

In [12]:
# create dummy variables
df_data = pd.get_dummies(df_Naplan)
df_data.head()

,CALENDAR_YEAR,YEAR_LEVEL,ENROLLED_STUDENTS,NMS,EXEMPT,ABSENT,ASSESSED,PARTICIPATION_RATE,PARTICIPATION_NUMBER,BANDCOL1,...,SUBGROUP_Parental Education: Year 11,SUBGROUP_Parental Education: Year 12,SUBGROUP_Parental Occupation: Group 1,SUBGROUP_Parental Occupation: Group 2,SUBGROUP_Parental Occupation: Group 3,SUBGROUP_Parental Occupation: Group 4,SUBGROUP_Parental Occupation: Not in paid work,SUBGROUP_Parental Occupation: Not stated,SUBGROUP_Remote,SUBGROUP_Very remote
689,2008,5,271423.0,91.9,1.5,2.7,95.0,96.9,263126,6.5,...,0,0,0,0,0,0,0,0,0,0
690,2008,5,41041.0,96.9,1.2,1.7,96.8,98.0,40233,1.9,...,0,0,0,0,0,0,0,0,0,0
691,2008,5,41794.0,92.0,1.4,1.9,96.4,97.8,40871,6.5,...,0,0,0,0,0,0,0,0,0,0
692,2008,5,22002.0,94.5,1.4,1.9,96.5,97.9,21547,4.1,...,0,0,0,0,0,0,0,0,0,0
693,2008,5,128634.0,91.8,1.2,3.5,94.9,96.2,123691,7.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Load and prepare data
X = df_Naplan.drop(columns=['SUBGROUP'])
y = df_Naplan['SUBGROUP']

In [15]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#Create Random Forest model
model = RandomForestClassifier(n_estimators=100,max_depth =10, random_state=42)

In [17]:
# Train the model
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'AUS'

In [ ]:
# Create Random Forest model
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

In [ ]:
# Define target vector
y = df_Naplan["SUBGROUP"].ravel()

In [ ]:
# Define features set
X = df_Naplan.copy()
X.head()
X.drop("SUBGROUP", axis=1, inplace=True)

In [ ]:
# selected_columns = ['CALENDAR_YEAR', 'YEAR_LEVEL',
#        'ENROLLED_STUDENTS', 'MEAN', 'MEAN_CI', 'MEAN_SD', 'NMS', 'NMS_CI',
#        'BANDCOL2', 'BANDCOL3', 'BANDCOL4', 'BANDCOL5', 'BANDCOL6', 'PCTL05',
#        'GAIN', 'GAIN_CI', 'EXEMPT', 'ABSENT', 'ASSESSED', 'WITHDRAWN',
#        'NONATTEMPT', 'PARTICIPATION_RATE', 'PARTICIPATION_NUMBER','BANDCOL1',
#        'PCTL20', 'PCTL80', 'PCTL95','TREND_VALUE']

In [ ]:

# TODO Convert categorical data to numeric with `pd.get_dummies`
# df = df.drop('CLASSIFICATION', axis=1)
# df_encoded
# df_encoded = pd.get_dummies(df)

In [ ]:
# Split our preprocessed data into our features and target arrays
#TODO X = df_encoded.drop(['IS_SUCCESSFUL'], axis=1)
# y = df_encoded['IS_SUCCESSFUL']

# # Splitting into Train and Test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()
# X_scaled = 
scaler.fit_transform(X)


In [ ]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Random Forest Model

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

## Making Predictions Using the Random Forest Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)